# Set up colab environment

In [ ]:
!git clone https://{GITHUB_ACCESS_TOKEN}@github.com/AliMohseninejad/ganbert-classifier.git
!rm ganbert-classifier/Codes/main.ipynb
!cp -r ganbert-classifier/Codes/data/ ./
!cp -r ganbert-classifier/Codes/evaluation/ ./
!cp -r ganbert-classifier/Codes/model/ ./
!cp -r ganbert-classifier/Codes/training/ ./
!cp -r ganbert-classifier/Dataset/ ../
!cp -r ganbert-classifier/Plots/ ../

In [ ]:
!pip install -qU transformers

# Vanilla BERT

In [ ]:
import torch

from data.data_loader import generate_dataloader
from model.bert import get_bert_model
from model.bert import get_tokenizer
from model.discriminator import Discriminator
from training.train import train_vanilla_classier

In [ ]:
learning_rate = 5e-5
model_name = "bert-base-cased"
bert_tokenizer, bert_config = get_tokenizer(model_name=model_name)

In [ ]:
for unsupervised_ratio in [0.99, 0.95, 0.90, 0.50]:
    bert_model, _ = get_bert_model(model_name=model_name)
    classifier = Discriminator()

    # Get dataloaders
    train_dataloader, val_dataloader, test_dataloader = generate_dataloader(
        dataset_folder_path="../Dataset/",
        unsupervised_ratio=unsupervised_ratio,
        tokenizer=bert_tokenizer,
        train_batch_size=4,
        valid_batch_size=4,
        test_batch_size=4,
        use_bow_dataset=False,
        random_seed=42,
    )

    # Define optimizer
    model_params = [v for v in bert_model.parameters()] + [
        v for v in classifier.parameters()
    ]
    optimizer = torch.optim.AdamW(model_params, lr=learning_rate)

    # Train the model
    bert_model, classifier, training_results = train_vanilla_classier(
        transformer=bert_model,
        classifier=classifier,
        optimizer=optimizer,
        epochs=10,
        scheduler=None,
        train_dataloader=train_dataloader,
        validation_dataloader=val_dataloader,
        save_path="../Plots/",
    )

    # Test the model

    # Visualize results
